# Setup

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
#import csvs from jupyter
trainPayments = pd.read_csv('df_Payments.csv')
trainOrderItems = pd.read_csv('df_OrderItems.csv')
trainCustomers = pd.read_csv('df_Customers.csv')
trainProducts = pd.read_csv('df_Products.csv')
trainOrders = pd.read_csv('df_Orders.csv')

testPayments = pd.read_csv('test_df_Payments.csv')
testOrderItems = pd.read_csv('test_df_OrderItems.csv')
testCustomers = pd.read_csv('test_df_Customers.csv')
testProducts = pd.read_csv('test_df_Products.csv')
testOrders = pd.read_csv('test_df_Orders.csv')

In [4]:
'''
#connect google drive
from google.colab import drive
drive.mount('/content/drive')

#import csvs from google drive
trainPayments = pd.read_csv('/content/drive/MyDrive/Ecommerce_Order_Dataset/train/df_Payments.csv')
trainOrderItems = pd.read_csv('/content/drive/MyDrive/Ecommerce_Order_Dataset/train/df_OrderItems.csv')
trainCustomers = pd.read_csv('/content/drive/MyDrive/Ecommerce_Order_Dataset/train/df_Customers.csv')
trainProducts = pd.read_csv('/content/drive/MyDrive/Ecommerce_Order_Dataset/train/df_Products.csv')
trainOrders = pd.read_csv('/content/drive/MyDrive/Ecommerce_Order_Dataset/train/df_Orders.csv')

testPayments = pd.read_csv('/content/drive/MyDrive/Ecommerce_Order_Dataset/test/df_Payments.csv')
testOrderItems = pd.read_csv('/content/drive/MyDrive/Ecommerce_Order_Dataset/test/df_OrderItems.csv')
testCustomers = pd.read_csv('/content/drive/MyDrive/Ecommerce_Order_Dataset/test/df_Customers.csv')
testProducts = pd.read_csv('/content/drive/MyDrive/Ecommerce_Order_Dataset/test/df_Products.csv')
testOrders = pd.read_csv('/content/drive/MyDrive/Ecommerce_Order_Dataset/test/df_Orders.csv')
'''

"\n#connect google drive\nfrom google.colab import drive\ndrive.mount('/content/drive')\n\n#import csvs from google drive\ntrainPayments = pd.read_csv('/content/drive/MyDrive/Ecommerce_Order_Dataset/train/df_Payments.csv')\ntrainOrderItems = pd.read_csv('/content/drive/MyDrive/Ecommerce_Order_Dataset/train/df_OrderItems.csv')\ntrainCustomers = pd.read_csv('/content/drive/MyDrive/Ecommerce_Order_Dataset/train/df_Customers.csv')\ntrainProducts = pd.read_csv('/content/drive/MyDrive/Ecommerce_Order_Dataset/train/df_Products.csv')\ntrainOrders = pd.read_csv('/content/drive/MyDrive/Ecommerce_Order_Dataset/train/df_Orders.csv')\n\ntestPayments = pd.read_csv('/content/drive/MyDrive/Ecommerce_Order_Dataset/test/df_Payments.csv')\ntestOrderItems = pd.read_csv('/content/drive/MyDrive/Ecommerce_Order_Dataset/test/df_OrderItems.csv')\ntestCustomers = pd.read_csv('/content/drive/MyDrive/Ecommerce_Order_Dataset/test/df_Customers.csv')\ntestProducts = pd.read_csv('/content/drive/MyDrive/Ecommerce_Orde

In [5]:
df = pd.concat([trainPayments, trainOrderItems, trainCustomers, trainProducts, trainOrders], axis=1) #merge training csvs to one dataframe
df = df.T.drop_duplicates().T #get rid of duplicate columns

In [6]:
test_df = pd.concat([testPayments, testOrderItems, testCustomers, testProducts, testOrders], axis=1) #merge testing csvs to one dataframe
test_df = test_df.T.drop_duplicates().T #get rid of duplicate columns

##Warning-- test_df does not have the last two columns of df, which give delivery time and delivery time prediction
##Split df into training and testing datasets in order to get accuracy

# Data cleaning/preprocessing


In [8]:
#Get rid of unneeded data
df = df.drop(columns=['order_id', 'product_id', 'seller_id', 'customer_id', 'customer_zip_code_prefix', 'customer_city', 'order_status', 'order_approved_at', 'order_estimated_delivery_date'])
test_df = test_df.drop(columns=['order_id', 'product_id', 'seller_id', 'customer_id', 'customer_zip_code_prefix', 'customer_city', 'order_approved_at', 'order_purchase_timestamp'])

In [9]:
#drops rows with NAN values
df = df.dropna()
test_df = test_df.dropna()

In [10]:
# Replace payment types with a vector that represents a certain payment type
payment_types = {'credit_card': 0, 'wallet': 1, 'voucher': 2, 'debit_card': 3}
x0 = []
for pay_type in list(df['payment_type']):
  pay_type_vector = [0,0,0,0]
  pay_type_vector[payment_types[pay_type]] = 1
  x0.append(pay_type_vector)
df['payment_type'] = x0

xt0 = []
for pay_type in list(test_df['payment_type']):
  pay_type_vector = [0,0,0,0]
  pay_type_vector[payment_types[pay_type]] = 1
  xt0.append(pay_type_vector)
test_df['payment_type'] = xt0

In [11]:
#Replace state with vector that represents state
customer_state = {
    'SP': 0, 'RJ': 1, 'MG': 2, 'SC': 3, 'ES': 4,
    'RN': 5, 'BA': 6, 'DF': 7, 'RS': 8, 'PE': 9,
    'GO': 10, 'CE': 11, 'PR': 12, 'MA': 13, 'PI': 14,
    'MT': 15, 'MS': 16, 'SE': 17, 'RO': 18, 'TO': 19,
    'AM': 20, 'AP': 21, 'PB': 22, 'PA': 23, 'AL': 24,
    'AC': 25, 'RR': 26
}
x1 = []
for state in list(df['customer_state']):
  customer_state_vector = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
  customer_state_vector[customer_state[state]] = 1
  x1.append(customer_state_vector)
df['customer_state'] = x1

xt1 = []
for state in list(test_df['customer_state']):
  customer_state_vector = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
  customer_state_vector[customer_state[state]] = 1
  xt1.append(customer_state_vector)
test_df['customer_state'] = xt1

In [12]:
# Put product categories into supercategories
home_and_living = [
    "furniture_decor",
    "bed_bath_table",
    "kitchen_dining_laundry_garden_furniture",
    "home_appliances",
    "home_comfort",
    "garden_tools",
    "housewares",
    "construction_tools_garden",
    "construction_tools_construction",
    "construction_tools_lights",
    "construction_tools_safety",
    "furniture_living_room",
    "furniture_bedroom",
    "furniture_mattress_and_upholstery"
]

fashion_and_accessories = [
    "fashion_shoes",
    "fashion_bags_accessories",
    "fashion_male_clothing",
    "fashio_female_clothing",
    "fashion_childrens_clothes",
    "fashion_underwear_beach",
    "fashion_sport"
]

toys_and_baby_products = [
    "toys",
    "baby",
    "diapers_and_hygiene"
]

health_and_beauty = [
    "health_beauty",
    "perfumery"
]

electronics_and_gadgets = [
    "audio",
    "computers_accessories",
    "computers",
    "consoles_games",
    "telephony",
    "small_appliances",
    "tablets_printing_image"
]

sports_and_leisure = [
    "sports_leisure"
]

food_and_beverages = [
    "food",
    "food_drink",
    "drinks"
]

arts_and_crafts = [
    "art",
    "arts_and_craftmanship"
]

gifts_and_celebrations = [
    "watches_gifts",
    "party_supplies",
    "christmas_supplies"
]

pets_and_animals = [
    "pet_shop"
]

office_and_stationery = [
    "office_furniture",
    "stationery"
]

industry_and_commerce = [
    "industry_commerce_and_business",
    "signaling_and_security",
    "agro_industry_and_commerce"
]

media_and_entertainment = [
    "books_technical",
    "books_general_interest",
    "books_imported",
    "dvds_blu_ray",
    "cine_photo",
    "music"
]

misc = [
    "cool_stuff"
]



cats0 = []

for cat in list(df['product_category_name']):
  if cat in home_and_living:
    cat = 'home_and_living'
  elif cat in fashion_and_accessories:
    cat = 'fashion_and_accessories'
  elif cat in toys_and_baby_products:
    cat = 'toys_and_baby_products'
  elif cat in health_and_beauty:
    cat = 'health_and_beauty'
  elif cat in electronics_and_gadgets:
    cat = 'electronics_and_gadgets'
  elif cat in sports_and_leisure:
    cat = 'sports_and_leisure'
  elif cat in food_and_beverages:
    cat = 'food_and_beverages'
  elif cat in arts_and_crafts:
    cat = 'arts_and_crafts'
  elif cat in gifts_and_celebrations:
    cat = 'gifts_and_celebrations'
  elif cat in pets_and_animals:
    cat = 'pets_and_animals'
  elif cat in office_and_stationery:
    cat = 'office_and_stationery'
  elif cat in industry_and_commerce:
    cat = 'industry_and_commerce'
  elif cat in media_and_entertainment:
    cat = 'media_and_entertainment'
  elif cat in misc:
    cat = 'misc'
  else:
    cat = 'misc'
  cats0.append(cat)

df['product_category_name'] = cats0



cats1 = []

for cat in list(test_df['product_category_name']):
  if cat in home_and_living:
    cat = 'home_and_living'
  elif cat in fashion_and_accessories:
    cat = 'fashion_and_accessories'
  elif cat in toys_and_baby_products:
    cat = 'toys_and_baby_products'
  elif cat in health_and_beauty:
    cat = 'health_and_beauty'
  elif cat in electronics_and_gadgets:
    cat = 'electronics_and_gadgets'
  elif cat in sports_and_leisure:
    cat = 'sports_and_leisure'
  elif cat in food_and_beverages:
    cat = 'food_and_beverages'
  elif cat in arts_and_crafts:
    cat = 'arts_and_crafts'
  elif cat in gifts_and_celebrations:
    cat = 'gifts_and_celebrations'
  elif cat in pets_and_animals:
    cat = 'pets_and_animals'
  elif cat in office_and_stationery:
    cat = 'office_and_stationery'
  elif cat in industry_and_commerce:
    cat = 'industry_and_commerce'
  elif cat in media_and_entertainment:
    cat = 'media_and_entertainment'
  elif cat in misc:
    cat = 'misc'
  else:
    cat = 'misc'
  cats1.append(cat)

test_df['product_category_name'] = cats1

In [13]:
#Replace product category with vector representing the product category
prod_cat = {
    'home_and_living': 0,
    'fashion_and_accessories': 1,
    'toys_and_baby_products': 2,
    'health_and_beauty': 3,
    'electronics_and_gadgets': 4,
    'sports_and_leisure': 5,
    'food_and_beverages': 6,
    'arts_and_crafts': 7,
    'gifts_and_celebrations': 8,
    'pets_and_animals': 9,
    'office_and_stationery': 10,
    'industry_and_commerce': 11,
    'media_and_entertainment': 12,
    'misc': 13
}

x2 = []
for cat in list(df['product_category_name']):
  prod_cat_vec = [0 for i in range(14)]
  prod_cat_vec[prod_cat[cat]] = 1
  x2.append(prod_cat_vec)
df['product_category_name'] = x2

xt2 = []
for cat in list(test_df['product_category_name']):
  prod_cat_vec = [0 for i in range(14)]
  prod_cat_vec[prod_cat[cat]] = 1
  xt2.append(prod_cat_vec)
test_df['product_category_name'] = xt2

In [14]:
# Convert timestamps to datetime format
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df['order_delivered_timestamp'] = pd.to_datetime(df['order_delivered_timestamp'])

# Find difference
df['delivery_time'] = df['order_delivered_timestamp'] - df['order_purchase_timestamp']

# Convert to hours
df['delivery_time_hours'] = df['delivery_time'].dt.total_seconds() / 3600

# Drop unnecessary columns
df.drop(columns=['order_purchase_timestamp', 'order_delivered_timestamp', 'delivery_time'], inplace=True)

In [15]:
# Features and labels
X = [] # Features
y = [] # Label
for i in range(len(df)):
    y.append(list(df['delivery_time_hours'])[i])
    
    X.append(
        [list(df['payment_sequential'])[i]] + 
        list(df['payment_type'])[i] +
        [list(df['payment_installments'])[i]] + 
        [list(df['payment_value'])[i]] +
        [list(df['price'])[i]] + 
        [list(df['shipping_charges'])[i]] +
        list(df['customer_state'])[i] + 
        list(df['product_category_name'])[i] +
        [list(df['product_weight_g'])[i]] + 
        [list(df['product_length_cm'])[i]] +
        [list(df['product_height_cm'])[i]] + 
        [list(df['product_width_cm'])[i]]
  )


KeyboardInterrupt: 

In [ ]:
#Train/test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#Scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Statistical Modeling

In [ ]:
# Create heatmap
columns = ['Payment Sequential',
    'Credit Card',
    'Digital Wallet',
    'Voucher',
    'Debit Card',
    'Payment Installments',
    'Payment Value',
    'Price',
    'Shipping Charges',
    'São Paulo (SP)',
    'Rio de Janeiro (RJ)',
    'Minas Gerais (MG)',
    'Santa Catarina (SC)',
    'Espírito Santo (ES)',
    'Rio Grande do Norte (RN)',
    'Bahia (BA)',
    'Distrito Federal (DF)',
    'Rio Grande do Sul (RS)',
    'Pernambuco (PE)',
    'Goiás (GO)',
    'Ceará (CE)',
    'Paraná (PR)',
    'Maranhão (MA)',
    'Piauí (PI)',
    'Mato Grosso (MT)',
    'Mato Grosso do Sul (MS)',
    'Sergipe (SE)',
    'Rondônia (RO)',
    'Tocantins (TO)',
    'Amazonas (AM)',
    'Amapá (AP)',
    'Paraíba (PB)',
    'Pará (PA)',
    'Alagoas (AL)',
    'Acre (AC)',
    'Roraima (RR)',
    'Home and Living',
    'Fashion and Accessories',
    'Toys and Baby Products',
    'Health and Beauty',
    'Electronics and Gadgets',
    'Sports and Leisure',
    'Food and Beverages',
    'Arts and Crafts',
    'Gifts and Celebrations',
    'Pets and Animals',
    'Office and Stationery',
    'Industry and Commerce',
    'Media and Entertainment',
    'Miscellaneous',
    'Product Weight (g)',
    'Product Length (cm)',
    'Product Height (cm)',
    'Product Width (cm)',
    'Delivery Time (hours)'
]

data_corr = pd.DataFrame(np.hstack([X, np.expand_dims(y,-1)]), columns=columns).corr()

labels = data_corr.copy()

sns.heatmap(data_corr, annot=True, xticklabels=data_corr.columns, yticklabels=data_corr.columns)
sns.set(rc = {'figure.figsize':(30,30)})
plt.show()

# ML Models

In [ ]:
!pip install tensorflow

In [ ]:
#Importing libraries
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras import Sequential
from tensorflow.keras import layers

In [ ]:
def getError(model, x_test, y_test):
    num_errors = 0
    tot_error = 0
    for j in range(x_test):
        tot_error += abs(model.predict(np.array([x_test[i]]))[0] - y_test[i])
        num_errors += 1
    return tot_error / num_errors

In [ ]:
#SVR model
error_df = {"i": [], "Error": []}
error = []

for i in range(1, 11):
    svr = SVR(degree=i)
    svr.fit(X_train_scaled, y_train)
    avg_error = getError(svr, X_test, y_test);
    error.append(avg_error)

error_df["Error"].append(error)
pd.DataFrame(error_df)
error_df

In [ ]:
#GBR model
error_df = {"i": [], "Error": []}
error = []

for i in range(1, 11):    
    gbr = GradientBoostingRegressor(n_estimators=i)
    gbr.fit(X_train, y_train)
    avg_error = getError(gbr, X_test, y_test)
    error.append(avg_error)

error_df["Error"].append(error)
pd.DataFrame(error_df)
error_df

In [ ]:
#Decision tree regressor
error_df = {"i": [], "Error": []}
error = []

for i in range(1, 11):    
    regressor = DecisionTreeRegressor(max_depth=i)
    regressor.fit(X_train, y_train)
    avg_error = getError(gbr, X_test, y_test)
    error.append(avg_error)

error_df["Error"].append(error)
pd.DataFrame(error_df)
error_df

In [ ]:
#Random forest
error_df = {"i": [], "Error": []}
error = []

for i in range(1, 11):
    regressor = RandomForestRegressor(n_estimators=i)
    regressor.fit(X_train, y_train)
    avg_error = getError(regressor, X_test, y_test)
    error.append(avg_error)

error_df["Error"].append(error)
pd.DataFrame(error_df)
error_df

In [ ]:
#Neural network
error_df = {"i": [], "Error": []}
error = []

def make_neural_network(num_hidden_layers):
    layers = [Input(55)] #input layer with 55 dimensions
    for i in range(num_hidden_layers): layers.append(Dense(100, activation="relu"))
    layers.append(Dense(1, activation="relu")) #output layer with one neuron
    
    model = Sequential(layers)
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

for i in range(1,11):
    model = make_neural_network(i)
    
    model.fit(X_train, y_train, epochs=300, batch_size=100, verbose=0)
    avg_error = getError(model, X_test, y_test)
    error.append(avg_error)

error_df["Error"].append(error)
pd.DataFrame(error_df)
error_df